### Table of Contents

### 00. Load Data
##### Data Selection

In [1]:
import random
import numpy as np
import cv2
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.optim as optim
from torch.nn.utils import clip_grad_value_

# for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed(123)

In [2]:
def get_random_list(num_items, start, end, set_seed=123):
    random.seed(set_seed)
    selected_idx = [i for i in range(start,end)]
    random.shuffle(selected_idx)
    return sorted(selected_idx[0:num_items])

def train_test_split(input_list, train_proportion=0.7, set_seed=123):
    train_idx = get_random_list(num_items=round(len(input_list)*train_proportion), 
                                start=0, end=len(input_list), set_seed=123)
    train_list = []
    test_list = []
    for i in range(0,len(input_list)):
        if i in train_idx:
            train_list.append(input_list[i])
        else:
            test_list.append(input_list[i])
    return train_list, test_list

In [3]:
# choose 25 PIE subjects
selected_idx = get_random_list(num_items=25, start=1, end=68, set_seed=123)
print('Selected subjects:', selected_idx)

Selected subjects: [2, 8, 13, 14, 15, 17, 19, 20, 23, 24, 27, 31, 32, 33, 34, 40, 42, 44, 46, 48, 50, 52, 59, 63, 65]


In [4]:
NUM_IMG_PER_SUBJ = 170
NUM_SELFIES = 10

# list of paths to PIE images
pie_list = []
for subj_idx in selected_idx:
    temp_list = ['PIE//'+str(subj_idx)+'//'+str(i+1)+'.jpg' for i in range(0,NUM_IMG_PER_SUBJ)]
    pie_list.extend(temp_list)

# list of paths to selfies
selfies_list = ['selfies//formatted//'+str(i+1)+'.jpg' for i in range(0,NUM_SELFIES)]

# list of paths to all images of interest
list_of_img_end_paths = pie_list + selfies_list

print('Number of selected PIE images:', len(pie_list))
print('Number of selected selfies:', len(selfies_list))
print('Number of selected images:', len(list_of_img_end_paths))

Number of selected PIE images: 4250
Number of selected selfies: 10
Number of selected images: 4260


##### Train Test Split

In [5]:
TRAIN_PROPORTION = 0.7

# split PIE train and test
pie_train_list, pie_test_list = train_test_split(pie_list, train_proportion=TRAIN_PROPORTION, set_seed=123)

# split selfies train and test
selfies_train_list, selfies_test_list = train_test_split(selfies_list, train_proportion=TRAIN_PROPORTION, set_seed=123)

print('Number||Proportion of train PIE images:', len(pie_train_list), '||', len(pie_train_list)/len(pie_list))
print('Number||Proportion of test PIE images:', len(pie_test_list), '||', len(pie_test_list)/len(pie_list))
print('Number||Proportion of train selfies:', len(selfies_train_list), '||', len(selfies_train_list)/len(selfies_list))
print('Number||Proportion of test selfies:', len(selfies_test_list), '||', len(selfies_test_list)/len(selfies_list))

Number||Proportion of train PIE images: 2975 || 0.7
Number||Proportion of test PIE images: 1275 || 0.3
Number||Proportion of train selfies: 7 || 0.7
Number||Proportion of test selfies: 3 || 0.3


In [6]:
def get_labels_from_path_list(list_of_img_paths, label_type='general'):
    label_list = []
    for path in list_of_img_paths:
        splitted = path.split('//')
        if splitted[0]=='PIE':
            if label_type=='general':
                label_list.append(splitted[0])
            elif label_type=='specific':
                label_list.append(splitted[1])
            else:
                print("Please specify label_type as 'general' or 'specific'")
        else:
            if label_type=='general':
                label_list.append(splitted[0])
            elif label_type=='specific':
                label_list.append('selfies')
            else:
                print("Please specify label_type as 'general' or 'specific'")
    return label_list

In [7]:
def load_and_vectorise_images(list_of_img_paths):
    # read image
    path = os.path.abspath('')

    for counter, end_paths in enumerate(list_of_img_paths):
        # load image
        img = cv2.imread(os.path.join(path, end_paths))

        # vectorise image
        array_size = img.shape[0]*img.shape[1]
        vectorised_img = img.copy()
        vectorised_img = vectorised_img.reshape(array_size,3)
        vectorised_img = np.array([i[0] for i in vectorised_img], dtype=int)

        # add to dataset
        if counter == 0:
            data = vectorised_img.reshape(1,array_size).copy()
        else:
            data = np.concatenate((data, vectorised_img.reshape(1,array_size)), axis=0)
    return data

In [8]:
# load test data
train_data = load_and_vectorise_images(pie_train_list+selfies_train_list)
test_data = load_and_vectorise_images(pie_test_list+selfies_test_list)
print('Vectorised and loaded train data:', train_data.shape)
print('Vectorised and loaded test data:', test_data.shape)

Vectorised and loaded train data: (2982, 1024)
Vectorised and loaded test data: (1278, 1024)


In [9]:
# prepare specific labels (target var for classification task)
train_labels = get_labels_from_path_list(pie_train_list+selfies_train_list, label_type='specific')
test_labels = get_labels_from_path_list(pie_test_list+selfies_test_list, label_type='specific')

In [10]:
# format labels into numeric values (& storing conversion dictionary)
def convert_str_to_num(str_list, str_to_num=None):
    unique = list(set(str_list))
    if str_to_num is None:
        str_to_num = {string:idx for idx, string in enumerate(unique)}
    else:
        missing = [i for i in unique if i not in str_to_num.keys()]
        max_idx = max(str_to_num.values())
        for string in missing:
            str_to_num[string]=max_idx
            max_idx=+1
        
    num_list = [str_to_num[string] for string in str_list]
    return num_list, str_to_num

train_labels_num, train_str_to_num = convert_str_to_num(train_labels)
test_labels_num, str_to_num = convert_str_to_num(test_labels, train_str_to_num)

In [11]:
# if false means we have unseen labels in test set, which might be problematic
assert train_str_to_num == str_to_num

### 05. Neural Network (NN)
Train a CNN with two convolutional
layers and one fully connected layer, with the architecture specified as follows: number of
nodes: 20-50-500-21. The number of the nodes in the last layer is fixed as 21 as we are performing
21-category (20 CMU PIE faces plus 1 for yourself) classification. Convolutional
kernel sizes are set as 5. Each convolutional layer is followed by a max pooling layer with
a kernel size of 2 and stride of 2. The fully connected layer is followed by ReLU. Train the
network and report the final classification performance.

##### NN for Classification

In [12]:
# constants
NUM_EPOCHS = 15
BATCH_SIZE = 10
LEARNING_RATE = 1e-2
CLIPPING_VALUE = 1
GENERATOR_PARAMS = {
    'shuffle': True,
    'num_workers': 0,
    'drop_last': True, #ignore last incomplete batch
    'pin_memory': True
}

# preferences
PRINT_ROUND = 50

# gpu
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    # print("Running on the GPU")
else:
    device = torch.device("cpu")
    # print("Running on the CPU")

In [13]:
# prepare data into pytorch tensor format
def prepare_sequence(seq):
    return torch.tensor(seq, dtype=torch.long)

class FormatDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        # Run multiple rows at once, i.e. reduce enumerate
        return len(self.features)

    def __getitem__(self, index):

        feats_in = prepare_sequence(self.features[index])
        target_out = prepare_sequence(self.labels[index])

        return feats_in, target_out
    
class CNN(nn.Module):
    def __init__(self, embed_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim, kernel_size, pool_k_size, pool_stride):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(embed_dim, hidden_dim1, kernel_size=kernel_size)
        self.conv2 = nn.Conv2d(hidden_dim1, hidden_dim2, kernel_size=kernel_size)
        self.pool = nn.MaxPool2d(kernel_size=pool_k_size, stride=pool_stride)
        self.fc1 = nn.Linear(hidden_dim2*5*5, hidden_dim3)
        self.fc2 = nn.Linear(hidden_dim3, output_dim)
        
    def forward(self, feats_in):
        batch_size, e_, e_, s_ = feats_in.shape
        x = self.pool(F.relu(self.conv1(feats_in.float())))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc2(x),dim=1)
        return x

In [14]:
def run():
    # load data
    training_set = FormatDataset(train_data, np.asarray(train_labels_num))
    training_generator = DataLoader(training_set, batch_size=BATCH_SIZE, **GENERATOR_PARAMS)

    test_batch_size = test_data.shape[0]
    test_set = FormatDataset(test_data, np.asarray(test_labels_num))
    testing_generator = DataLoader(test_set, batch_size=test_batch_size, **GENERATOR_PARAMS)

    # load model
    model = CNN(embed_dim=1,
                hidden_dim1=20, 
                hidden_dim2=50,
                hidden_dim3=500,
                output_dim=26, 
                kernel_size=(5,5),
                pool_k_size=(2,2),
                pool_stride=2
               ).to(device)
    loss_function = nn.CrossEntropyLoss().to(device)
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

    results_dict = {}
    early_stopping = False

    # train
    for epoch in range(NUM_EPOCHS):
        print('Epoch {}'.format(epoch), '-'*80)
        results = {'train_loss': [], 'train_acc': []}
        for idx, (feats_in, target_out) in enumerate(training_generator):
            feats_in = feats_in.to(device).view(BATCH_SIZE, 32, 32).unsqueeze(1)
            target_out = target_out.to(device)
            model.zero_grad()
            tag_scores = model(feats_in).to(device)
            predicted = torch.argmax(tag_scores, dim=1).detach()
            correct = sum([1 for pred,act in zip(predicted,target_out) if pred==act])
            accuracy = correct/len(target_out)
            loss = loss_function(tag_scores, target_out)
            loss.backward()
            clip_grad_value_(model.parameters(), CLIPPING_VALUE)
            optimizer.step()

            results['train_loss'].append(loss.data.item())
            results['train_acc'].append(accuracy)

            if idx%PRINT_ROUND==0:
                print('Step {} | Training Loss: {}, Accuracy: {}%'.format(idx, round(loss.data.item(),3), round(accuracy*100,3)))

        results_dict[epoch] = results
        avg_acc = sum(results['train_acc'])/(idx+1)
        print('Average Accuracy: {}%'.format(round(avg_acc*100,3)))

    # predict
    with torch.no_grad():
        print('-'*80)
        for idx, (feats_in, target_out) in enumerate(testing_generator):
            feats_in = feats_in.to(device).view(test_batch_size, 32, 32).unsqueeze(1)
            target_out = target_out.to(device)
            tag_scores = model(feats_in).to(device)
            predicted = torch.argmax(tag_scores, dim=1).detach()
            correct = sum([1 for pred,act in zip(predicted,target_out) if pred==act])
            accuracy = correct/test_batch_size
            print('Testing Accuracy: {}%'.format(round(accuracy*100,3)))
            
    # print results || selfies idx is str_to_num['selfies']
    correct_selfies = sum([1 for pred,act in zip(predicted,target_out) if (pred==act) & (act==str_to_num['selfies'])])
    correct_PIE = correct - correct_selfies
    total_selfies = 3
    total_PIE = test_batch_size-total_selfies
    print('Accuracy of NN for ALL test set: {}%'.format(round(accuracy*100,3)))
    print('Accuracy of NN for PIE test set: {}%'.format(round((correct_PIE/total_PIE)*100,3)))
    print('Accuracy of NN for selfies test set: {}%'.format(round((correct_selfies/total_selfies)*100,3)))
            
run()

Epoch 0 --------------------------------------------------------------------------------
Step 0 | Training Loss: 13.633, Accuracy: 0.0%
Step 50 | Training Loss: 3.26, Accuracy: 0.0%
Step 100 | Training Loss: 3.274, Accuracy: 10.0%
Step 150 | Training Loss: 3.26, Accuracy: 0.0%
Step 200 | Training Loss: 3.19, Accuracy: 0.0%
Step 250 | Training Loss: 3.575, Accuracy: 0.0%
Average Accuracy: 6.913%
Epoch 1 --------------------------------------------------------------------------------
Step 0 | Training Loss: 2.805, Accuracy: 30.0%
Step 50 | Training Loss: 2.023, Accuracy: 40.0%
Step 100 | Training Loss: 3.275, Accuracy: 10.0%
Step 150 | Training Loss: 0.762, Accuracy: 80.0%
Step 200 | Training Loss: 0.323, Accuracy: 90.0%
Step 250 | Training Loss: 2.103, Accuracy: 50.0%
Average Accuracy: 56.074%
Epoch 2 --------------------------------------------------------------------------------
Step 0 | Training Loss: 0.299, Accuracy: 100.0%
Step 50 | Training Loss: 0.747, Accuracy: 80.0%
Step 100 | 

##### Experimenting with different network architectures

In [15]:
# 1. Without ReLU between CNN Layers (failed model)
class CNN(nn.Module):
    def __init__(self, embed_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim, kernel_size, pool_k_size, pool_stride):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(embed_dim, hidden_dim1, kernel_size=kernel_size)
        self.conv2 = nn.Conv2d(hidden_dim1, hidden_dim2, kernel_size=kernel_size)
        self.pool = nn.MaxPool2d(kernel_size=pool_k_size, stride=pool_stride)
        self.fc1 = nn.Linear(hidden_dim2*5*5, hidden_dim3)
        self.fc2 = nn.Linear(hidden_dim3, output_dim)
        
    def forward(self, feats_in):
        batch_size, e_, e_, s_ = feats_in.shape
        x = self.pool(self.conv1(feats_in.float()))
        x = self.pool(self.conv2(x))
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc2(x),dim=1)
        return x

run()

Epoch 0 --------------------------------------------------------------------------------
Step 0 | Training Loss: 13.595, Accuracy: 10.0%
Step 50 | Training Loss: 122.357, Accuracy: 0.0%
Step 100 | Training Loss: 3.253, Accuracy: 20.0%
Step 150 | Training Loss: 3.261, Accuracy: 10.0%
Step 200 | Training Loss: 3.256, Accuracy: 0.0%
Step 250 | Training Loss: 3.266, Accuracy: 0.0%
Average Accuracy: 4.329%
Epoch 1 --------------------------------------------------------------------------------
Step 0 | Training Loss: 3.261, Accuracy: 0.0%
Step 50 | Training Loss: 3.265, Accuracy: 0.0%
Step 100 | Training Loss: 3.267, Accuracy: 0.0%
Step 150 | Training Loss: 3.26, Accuracy: 0.0%
Step 200 | Training Loss: 3.26, Accuracy: 0.0%
Step 250 | Training Loss: 3.257, Accuracy: 10.0%
Average Accuracy: 3.523%
Epoch 2 --------------------------------------------------------------------------------
Step 0 | Training Loss: 3.248, Accuracy: 0.0%
Step 50 | Training Loss: 3.247, Accuracy: 10.0%
Step 100 | Tra

In [16]:
# 2. Reducing batch sizes
BATCH_SIZE=5

# reset CNN
class CNN(nn.Module):
    def __init__(self, embed_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim, kernel_size, pool_k_size, pool_stride):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(embed_dim, hidden_dim1, kernel_size=kernel_size)
        self.conv2 = nn.Conv2d(hidden_dim1, hidden_dim2, kernel_size=kernel_size)
        self.pool = nn.MaxPool2d(kernel_size=pool_k_size, stride=pool_stride)
        self.fc1 = nn.Linear(hidden_dim2*5*5, hidden_dim3)
        self.fc2 = nn.Linear(hidden_dim3, output_dim)
        
    def forward(self, feats_in):
        batch_size, e_, e_, s_ = feats_in.shape
        x = self.pool(F.relu(self.conv1(feats_in.float())))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc2(x),dim=1)
        return x

run()

Epoch 0 --------------------------------------------------------------------------------
Step 0 | Training Loss: 8.254, Accuracy: 0.0%
Step 50 | Training Loss: 3.091, Accuracy: 0.0%
Step 100 | Training Loss: 1.945, Accuracy: 60.0%
Step 150 | Training Loss: 1.349, Accuracy: 40.0%
Step 200 | Training Loss: 1.066, Accuracy: 80.0%
Step 250 | Training Loss: 1.206, Accuracy: 60.0%
Step 300 | Training Loss: 0.945, Accuracy: 60.0%
Step 350 | Training Loss: 2.114, Accuracy: 60.0%
Step 400 | Training Loss: 0.722, Accuracy: 80.0%
Step 450 | Training Loss: 0.99, Accuracy: 60.0%
Step 500 | Training Loss: 5.086, Accuracy: 0.0%
Step 550 | Training Loss: 1.144, Accuracy: 60.0%
Average Accuracy: 50.268%
Epoch 1 --------------------------------------------------------------------------------
Step 0 | Training Loss: 0.031, Accuracy: 100.0%
Step 50 | Training Loss: 0.024, Accuracy: 100.0%
Step 100 | Training Loss: 0.072, Accuracy: 100.0%
Step 150 | Training Loss: 1.081, Accuracy: 60.0%
Step 200 | Training

Step 450 | Training Loss: 0.0, Accuracy: 100.0%
Step 500 | Training Loss: 0.0, Accuracy: 100.0%
Step 550 | Training Loss: 0.0, Accuracy: 100.0%
Average Accuracy: 99.664%
Epoch 12 --------------------------------------------------------------------------------
Step 0 | Training Loss: 0.0, Accuracy: 100.0%
Step 50 | Training Loss: 0.0, Accuracy: 100.0%
Step 100 | Training Loss: 0.0, Accuracy: 100.0%
Step 150 | Training Loss: 0.0, Accuracy: 100.0%
Step 200 | Training Loss: 0.0, Accuracy: 100.0%
Step 250 | Training Loss: 0.0, Accuracy: 100.0%
Step 300 | Training Loss: 0.0, Accuracy: 100.0%
Step 350 | Training Loss: 0.0, Accuracy: 100.0%
Step 400 | Training Loss: 0.0, Accuracy: 100.0%
Step 450 | Training Loss: 0.0, Accuracy: 100.0%
Step 500 | Training Loss: 0.0, Accuracy: 100.0%
Step 550 | Training Loss: 0.0, Accuracy: 100.0%
Average Accuracy: 99.966%
Epoch 13 --------------------------------------------------------------------------------
Step 0 | Training Loss: 0.0, Accuracy: 100.0%
Step 

In [17]:
# 2. Increasing batch sizes
BATCH_SIZE=50

run()

Epoch 0 --------------------------------------------------------------------------------
Step 0 | Training Loss: 8.901, Accuracy: 6.0%
Step 50 | Training Loss: 3.244, Accuracy: 4.0%
Average Accuracy: 4.881%
Epoch 1 --------------------------------------------------------------------------------
Step 0 | Training Loss: 3.237, Accuracy: 4.0%
Step 50 | Training Loss: 3.165, Accuracy: 14.0%
Average Accuracy: 7.763%
Epoch 2 --------------------------------------------------------------------------------
Step 0 | Training Loss: 3.037, Accuracy: 12.0%
Step 50 | Training Loss: 2.252, Accuracy: 34.0%
Average Accuracy: 30.949%
Epoch 3 --------------------------------------------------------------------------------
Step 0 | Training Loss: 1.541, Accuracy: 54.0%
Step 50 | Training Loss: 0.545, Accuracy: 88.0%
Average Accuracy: 72.881%
Epoch 4 --------------------------------------------------------------------------------
Step 0 | Training Loss: 0.426, Accuracy: 88.0%
Step 50 | Training Loss: 0.28

In [18]:
# 2. Increasing batch sizes | with early stopping criteria
BATCH_SIZE=128
NUM_EPOCHS=300

# load data
training_set = FormatDataset(train_data, np.asarray(train_labels_num))
training_generator = DataLoader(training_set, batch_size=BATCH_SIZE, **GENERATOR_PARAMS)

test_batch_size = test_data.shape[0]
test_set = FormatDataset(test_data, np.asarray(test_labels_num))
testing_generator = DataLoader(test_set, batch_size=test_batch_size, **GENERATOR_PARAMS)

# load model
model = CNN(embed_dim=1,
            hidden_dim1=20, 
            hidden_dim2=50,
            hidden_dim3=500,
            output_dim=26, 
            kernel_size=(5,5),
            pool_k_size=(2,2),
            pool_stride=2
           ).to(device)
loss_function = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

results_dict = {}
early_stopping = False

# train
for epoch in range(NUM_EPOCHS):
#     print('Epoch {}'.format(epoch), '-'*80)
    results = {'train_loss': [], 'train_acc': []}
    for idx, (feats_in, target_out) in enumerate(training_generator):
        feats_in = feats_in.to(device).view(BATCH_SIZE, 32, 32).unsqueeze(1)
        target_out = target_out.to(device)
        model.zero_grad()
        tag_scores = model(feats_in).to(device)
        predicted = torch.argmax(tag_scores, dim=1).detach()
        correct = sum([1 for pred,act in zip(predicted,target_out) if pred==act])
        accuracy = correct/len(target_out)
        loss = loss_function(tag_scores, target_out)
        loss.backward()
        optimizer.step()

        results['train_loss'].append(loss.data.item())
        results['train_acc'].append(accuracy)

    if epoch%10==0:
        print('Epoch {} | Training Loss: {}, Accuracy: {}%'.format(epoch, round(loss.data.item(),3), round(accuracy*100,3)))
        if accuracy>0.99:
            break

    results_dict[epoch] = results
    avg_acc = sum(results['train_acc'])/(idx+1)
#     print('Average Accuracy: {}%'.format(round(avg_acc*100,3)))
    
# predict
with torch.no_grad():
    print('-'*80)
    for idx, (feats_in, target_out) in enumerate(testing_generator):
        feats_in = feats_in.to(device).view(test_batch_size, 32, 32).unsqueeze(1)
        target_out = target_out.to(device)
        tag_scores = model(feats_in).to(device)
        predicted = torch.argmax(tag_scores, dim=1).detach()
        correct = sum([1 for pred,act in zip(predicted,target_out) if pred==act])
        accuracy = correct/test_batch_size
        print('Testing Accuracy: {}%'.format(round(accuracy*100,3)))
        
# print results || selfies idx is str_to_num['selfies']
correct_selfies = sum([1 for pred,act in zip(predicted,target_out) if (pred==act) & (act==str_to_num['selfies'])])
correct_PIE = correct - correct_selfies
total_selfies = 3
total_PIE = test_batch_size-total_selfies
print('Accuracy of NN for ALL test set: {}%'.format(round(accuracy*100,3)))
print('Accuracy of NN for PIE test set: {}%'.format(round((correct_PIE/total_PIE)*100,3)))
print('Accuracy of NN for selfies test set: {}%'.format(round((correct_selfies/total_selfies)*100,3)))

Epoch 0 | Training Loss: 3.278, Accuracy: 3.125%
Epoch 10 | Training Loss: 3.253, Accuracy: 3.125%
Epoch 20 | Training Loss: 3.194, Accuracy: 9.375%
Epoch 30 | Training Loss: 2.959, Accuracy: 14.062%
Epoch 40 | Training Loss: 1.146, Accuracy: 63.281%
Epoch 50 | Training Loss: 0.148, Accuracy: 97.656%
Epoch 60 | Training Loss: 0.021, Accuracy: 100.0%
--------------------------------------------------------------------------------
Testing Accuracy: 95.696%
Accuracy of NN for ALL test set: 95.696%
Accuracy of NN for PIE test set: 95.843%
Accuracy of NN for selfies test set: 33.333%


In [19]:
# 3. Comments on reproducibility of accuracies: REMOVE gradient clipping

# revert values
BATCH_SIZE=10
NUM_EPOCHS=15

def run1():
    # load data
    training_set = FormatDataset(train_data, np.asarray(train_labels_num))
    training_generator = DataLoader(training_set, batch_size=BATCH_SIZE, **GENERATOR_PARAMS)

    test_batch_size = test_data.shape[0]
    test_set = FormatDataset(test_data, np.asarray(test_labels_num))
    testing_generator = DataLoader(test_set, batch_size=test_batch_size, **GENERATOR_PARAMS)

    # load model
    model = CNN(embed_dim=1,
                hidden_dim1=20, 
                hidden_dim2=50,
                hidden_dim3=500,
                output_dim=26, 
                kernel_size=(5,5),
                pool_k_size=(2,2),
                pool_stride=2
               ).to(device)
    loss_function = nn.CrossEntropyLoss().to(device)
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

    results_dict = {}
    early_stopping = False

    # train
    for epoch in range(NUM_EPOCHS):
        print('Epoch {}'.format(epoch), '-'*80)
        results = {'train_loss': [], 'train_acc': []}
        for idx, (feats_in, target_out) in enumerate(training_generator):
            feats_in = feats_in.to(device).view(BATCH_SIZE, 32, 32).unsqueeze(1)
            target_out = target_out.to(device)
            model.zero_grad()
            tag_scores = model(feats_in).to(device)
            predicted = torch.argmax(tag_scores, dim=1).detach()
            correct = sum([1 for pred,act in zip(predicted,target_out) if pred==act])
            accuracy = correct/len(target_out)
            loss = loss_function(tag_scores, target_out)
            loss.backward()
#             clip_grad_value_(model.parameters(), CLIPPING_VALUE)
            optimizer.step()

            results['train_loss'].append(loss.data.item())
            results['train_acc'].append(accuracy)

            if idx%PRINT_ROUND==0:
                print('Step {} | Training Loss: {}, Accuracy: {}%'.format(idx, round(loss.data.item(),3), round(accuracy*100,3)))

        results_dict[epoch] = results
        avg_acc = sum(results['train_acc'])/(idx+1)
        print('Average Accuracy: {}%'.format(round(avg_acc*100,3)))

    # predict
    with torch.no_grad():
        print('-'*80)
        for idx, (feats_in, target_out) in enumerate(testing_generator):
            feats_in = feats_in.to(device).view(test_batch_size, 32, 32).unsqueeze(1)
            target_out = target_out.to(device)
            tag_scores = model(feats_in).to(device)
            predicted = torch.argmax(tag_scores, dim=1).detach()
            correct = sum([1 for pred,act in zip(predicted,target_out) if pred==act])
            accuracy = correct/test_batch_size
            print('Testing Accuracy: {}%'.format(round(accuracy*100,3)))
            
    # print results || selfies idx is str_to_num['selfies']
    correct_selfies = sum([1 for pred,act in zip(predicted,target_out) if (pred==act) & (act==str_to_num['selfies'])])
    correct_PIE = correct - correct_selfies
    total_selfies = 3
    total_PIE = test_batch_size-total_selfies
    print('Accuracy of NN for ALL test set: {}%'.format(round(accuracy*100,3)))
    print('Accuracy of NN for PIE test set: {}%'.format(round((correct_PIE/total_PIE)*100,3)))
    print('Accuracy of NN for selfies test set: {}%'.format(round((correct_selfies/total_selfies)*100,3)))

set_seed(123)
run1()
set_seed(234)
run1()
set_seed(345)
run1()

Epoch 0 --------------------------------------------------------------------------------
Step 0 | Training Loss: 13.633, Accuracy: 0.0%
Step 50 | Training Loss: 3.299, Accuracy: 0.0%
Step 100 | Training Loss: 3.242, Accuracy: 10.0%
Step 150 | Training Loss: 3.263, Accuracy: 0.0%
Step 200 | Training Loss: 3.257, Accuracy: 0.0%
Step 250 | Training Loss: 3.259, Accuracy: 0.0%
Average Accuracy: 4.027%
Epoch 1 --------------------------------------------------------------------------------
Step 0 | Training Loss: 3.255, Accuracy: 0.0%
Step 50 | Training Loss: 3.234, Accuracy: 20.0%
Step 100 | Training Loss: 3.247, Accuracy: 20.0%
Step 150 | Training Loss: 3.219, Accuracy: 0.0%
Step 200 | Training Loss: 3.252, Accuracy: 0.0%
Step 250 | Training Loss: 3.232, Accuracy: 0.0%
Average Accuracy: 3.859%
Epoch 2 --------------------------------------------------------------------------------
Step 0 | Training Loss: 3.235, Accuracy: 10.0%
Step 50 | Training Loss: 3.26, Accuracy: 0.0%
Step 100 | Train

Step 250 | Training Loss: 0.002, Accuracy: 100.0%
Average Accuracy: 98.557%
Epoch 5 --------------------------------------------------------------------------------
Step 0 | Training Loss: 0.012, Accuracy: 100.0%
Step 50 | Training Loss: 0.0, Accuracy: 100.0%
Step 100 | Training Loss: 0.002, Accuracy: 100.0%
Step 150 | Training Loss: 0.006, Accuracy: 100.0%
Step 200 | Training Loss: 0.003, Accuracy: 100.0%
Step 250 | Training Loss: 0.004, Accuracy: 100.0%
Average Accuracy: 99.899%
Epoch 6 --------------------------------------------------------------------------------
Step 0 | Training Loss: 0.001, Accuracy: 100.0%
Step 50 | Training Loss: 0.004, Accuracy: 100.0%
Step 100 | Training Loss: 0.001, Accuracy: 100.0%
Step 150 | Training Loss: 0.0, Accuracy: 100.0%
Step 200 | Training Loss: 0.0, Accuracy: 100.0%
Step 250 | Training Loss: 0.004, Accuracy: 100.0%
Average Accuracy: 100.0%
Epoch 7 --------------------------------------------------------------------------------
Step 0 | Training 

Step 150 | Training Loss: 0.03, Accuracy: 100.0%
Step 200 | Training Loss: 0.007, Accuracy: 100.0%
Step 250 | Training Loss: 0.028, Accuracy: 100.0%
Average Accuracy: 93.557%
Epoch 10 --------------------------------------------------------------------------------
Step 0 | Training Loss: 0.002, Accuracy: 100.0%
Step 50 | Training Loss: 0.16, Accuracy: 90.0%
Step 100 | Training Loss: 0.049, Accuracy: 100.0%
Step 150 | Training Loss: 0.0, Accuracy: 100.0%
Step 200 | Training Loss: 0.222, Accuracy: 90.0%
Step 250 | Training Loss: 0.319, Accuracy: 90.0%
Average Accuracy: 93.356%
Epoch 11 --------------------------------------------------------------------------------
Step 0 | Training Loss: 0.022, Accuracy: 100.0%
Step 50 | Training Loss: 0.107, Accuracy: 100.0%
Step 100 | Training Loss: 0.001, Accuracy: 100.0%
Step 150 | Training Loss: 0.008, Accuracy: 100.0%
Step 200 | Training Loss: 0.213, Accuracy: 90.0%
Step 250 | Training Loss: 0.002, Accuracy: 100.0%
Average Accuracy: 97.081%
Epoch 